In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset

In [4]:
lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [5]:
lc_quad_train[0]["graph"]

[['?x',
  '<http://dbpedia.org/property/operator>',
  '<http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide>'],
 ['?x', '<http://dbpedia.org/property/address>', '?uri']]

In [16]:
from query.query_functions2 import run_sparql_query, count_from_triples, make_triplet

In [28]:
for i, record in enumerate(lc_quad_train):

    type_triples = list(filter(lambda x: "http://www.w3.org/1999/02/22-rdf-syntax-ns#type" in x[1], record["graph"]))
    if len(type_triples) == 0:
        continue

    for i in range(len(type_triples)):
        for j, element in enumerate(type_triples[i]):
            if element.startswith("?"):
                type_triples[i][j] = "?uri"
    type_count = [run_sparql_query(f"SELECT count(?uri) WHERE {{ {' '.join(ttrip)} }}")[0]["callret-0"]["value"] for ttrip in type_triples]
    print(i, type_count)


0 ['76640']
0 ['10391']
0 ['78119']
0 ['706']
0 ['65333']
0 ['1767272']
0 ['13547']
0 ['146449']
0 ['28162']
0 ['78119']
0 ['174970']
0 ['1388']
0 ['13547']
0 ['120']
0 ['146449']
0 ['78119']
0 ['21684']
0 ['528']
0 ['23015']
0 ['1767272']
0 ['13547']
0 ['146449']
0 ['1767272']
0 ['53836']
0 ['146449']
0 ['52440']
0 ['65333']
0 ['11131']
0 ['13547']
0 ['99745']
0 ['220925']
0 ['1767272']
0 ['146449']
0 ['1767272']
0 ['118415']
0 ['99745']
0 ['65333']
0 ['6054']
0 ['12758']
0 ['7760']
0 ['11430']
0 ['1767272']
0 ['65333']
0 ['4209']
0 ['39039']
0 ['38870']
0 ['17654']
0 ['146449']
0 ['68254']
0 ['146449']
0 ['78119']
0 ['55179']
0 ['23173']
0 ['6290']
0 ['36029']
0 ['220925']
0 ['65333']
0 ['5025']
0 ['146449']
0 ['174970']
0 ['1767272']
0 ['1388']
0 ['146449']
0 ['65333']
0 ['146449']
0 ['4209']
0 ['6290']
0 ['146449']
0 ['13779']
0 ['19067']
0 ['13547']
0 ['174970']
0 ['1770']
0 ['1767272']
0 ['816252']
0 ['1767272']
0 ['1767272']
0 ['146449']
0 ['146449']
0 ['146449']
0 ['11430']
0 [

In [26]:
run_sparql_query(f"SELECT count(?uri) WHERE {{ {' '.join(type_triples[0])} }}")

[{'callret-0': {'type': 'typed-literal',
   'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'value': '65333'}}]

In [71]:
from helper.query_helpers import get_all_nodes_touched, get_neighbourhood
from evaluation.evaluate_script import filter_url

In [67]:
lc_quad_train[0]["graph"]

[['?x',
  '<http://dbpedia.org/property/operator>',
  '<http://dbpedia.org/resource/Starwood_Hotels_and_Resorts_Worldwide>'],
 ['?x', '<http://dbpedia.org/property/address>', '?uri']]

In [72]:
query_nodes = get_all_nodes_touched(lc_quad_train[0]["graph"])
query_nodes_filtered = list(filter(lambda x: filter_url(x), query_nodes))

In [73]:
neighbourhood_connections = get_neighbourhood(query_nodes_filtered)

In [74]:
query_nodes_filtered

['http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel',
 'http://dbpedia.org/resource/Palace_Hotel,_San_Francisco',
 'http://dbpedia.org/resource/Chennai',
 'http://dbpedia.org/resource/Tamil_Nadu',
 'http://dbpedia.org/resource/Westminster,_Colorado',
 'http://dbpedia.org/resource/Westin_Westminster',
 'http://dbpedia.org/resource/San_Francisco',
 'http://dbpedia.org/resource/San_Jose,_California',
 'http://dbpedia.org/resource/Hotel_Montgomery',
 'http://dbpedia.org/resource/Dhaka_Westin',
 'http://dbpedia.org/resource/California',
 'http://dbpedia.org/resource/Hotel_Bristol,_Warsaw',
 'http://dbpedia.org/resource/Hotel_Alfonso_XIII',
 'http://dbpedia.org/resource/Westin_Seattle',
 'http://dbpedia.org/resource/Le_Méridien_San_Francisco',
 'http://dbpedia.org/resource/Seattle',
 'http://dbpedia.org/resource/Westin_Peachtree_Plaza_Hotel',
 'http://dbpedia.org/resource/Le_Royal_Meridien,_Chennai',
 'http://dbpedia.org/resource/W_San_Francisco',
 'http://dbpedia.org/resource

In [132]:
all_nodes = set(map(lambda x: x[0], neighbourhood_connections)).union(set(map(lambda x: x[2], neighbourhood_connections)))
print("edges", len(neighbourhood_connections))
print("nodes", len(all_nodes))

edges 205716
nodes 120307


In [120]:
neighbourhood_connections_edgelist = ["###".join([triple[0],
                                                  triple[2],
                                                  triple[1]]) for triple in neighbourhood_connections]

# neighbourhood_connections_edgelist = [" ".join([triple[0].replace(" ", "_").strip(),
#                                                 triple[2].replace(" ", "_").strip(),
#                                                 triple[1].replace(" ", "_").strip()]) for triple in neighbourhood_connections]

In [108]:
neighbourhood_connections_edgelist[:5]

['http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel Sheraton_New_York_Times_Square_Hotel http://xmlns.com/foaf/0.1/name',
 'http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel http://en.wikipedia.org/wiki/Sheraton_New_York_Times_Square_Hotel http://xmlns.com/foaf/0.1/isPrimaryTopicOf',
 'http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel http://www.sheratonnewyork.com http://xmlns.com/foaf/0.1/homepage',
 'http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel http://commons.wikimedia.org/wiki/Special:FilePath/Sheraton_NY_H&T_west_jeh.jpg http://xmlns.com/foaf/0.1/depiction',
 'http://dbpedia.org/resource/Sheraton_New_York_Times_Square_Hotel http://en.wikipedia.org/wiki/Sheraton_New_York_Times_Square_Hotel?oldid=695685468 http://www.w3.org/ns/prov#wasDerivedFrom']

In [125]:
import networkx as nx

In [139]:
# G = nx.parse_edgelist(neighbourhood_connections_edgelist, nodetype=str, data=(("label", str),), delimiter="###")
G = nx.Graph()

G.add_nodes_from(all_nodes)

for conn in neighbourhood_connections:
    G.add_edge(conn[0], conn[2], label=conn[1])
    # G.add_edge(conn[0], conn[2])


In [140]:
nx.degree_centrality(G)

{'': 8.312137383006666e-06,
 'http://dbpedia.org/resource/1939': 8.312137383006666e-06,
 'http://www.hindu.com/br/2007/04/03/stories/2007040300301600.htm': 8.312137383006666e-06,
 'http://dbpedia.org/resource/Closing_Time_(album)': 8.312137383006666e-06,
 'http://dbpedia.org/resource/1853_in_the_United_States': 1.6624274766013333e-05,
 'http://dbpedia.org/resource/Venice:_The_Series': 8.312137383006666e-06,
 'http://dbpedia.org/resource/Gudakkarai': 8.312137383006666e-06,
 'http://dbpedia.org/resource/Thomas_Milton_Gatch': 1.6624274766013333e-05,
 'http://dbpedia.org/resource/K._Kunchunniraja': 8.312137383006666e-06,
 'http://dbpedia.org/resource/KGMI': 8.312137383006666e-06,
 'http://dbpedia.org/resource/N_K_colony': 8.312137383006666e-06,
 'http://dbpedia.org/resource/Again_(Alice_in_Chains_song)': 8.312137383006666e-06,
 "http://dbpedia.org/resource/Salim_Ali's_fruit_bat": 8.312137383006666e-06,
 'http://dbpedia.org/resource/2005_WGC-American_Express_Championship': 8.312137383006666

In [141]:
print(len(G.nodes))
print(len(G.edges))

120307
138860
